In [1]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Name 1: Neddy N
- Name 2: Blake H
- Name 3: Rohit C

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [3]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.1214353154362896 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 
##### 11000
- What does each instance represent? 
##### Article
- How many classes are in the dataset and what does each class represent?
##### 20 , they represent a newsgroup

- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
##### No. Because some of the articles are dated (20 years)
- Is the data represented as a sparse or dense matrix?
##### sparse

___
Enter your answer here:

*Double click to edit*





___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [29]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np
# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)
# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)
# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)
# Enter distance comparison below for each pair of vectors:
p = 'Placeholder'
print('\n\nEuclidean Distance\n ab:', euclidean(a,b), 'ac:', euclidean(a,c), 'bc:',euclidean(b,c))
print('Cosine Distance\n ab:', cosine(a,b), 'ac:', cosine(a,c), 'bc:',cosine(b,c))
print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:', jaccard(a,b), 'ac:', jaccard(a,c), 'bc:',jaccard(b,c))
print('\n\nThe most appropriate distance is...')
print('Cosine distance shows the greatest distances between topics and gives us the best differentiation ability. It is the best distance metric for this analysis.')

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.0985184671870858 ac: 1.1891405425398234 bc: 0.9177794226661624
Cosine Distance
 ab: 0.6033714113755322 ac: 0.7070276149559529 bc: 0.4211595343347173
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 1.0 ac: 1.0 bc: 1.0


The most appropriate distance is...
Cosine distance shows the greatest distances between topics and gives us the best differentiation ability. It is the best distance metric for this analysis.


___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [27]:
import sklearn.model_selection as ms
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 
from sklearn import metrics as mt
from sklearn.metrics import (accuracy_score,brier_score_loss, precision_score, recall_score,f1_score)
import pandas as pd
import numpy as ny
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.express as px
import os 

cv = ms.StratifiedShuffleSplit( n_splits = 1, test_size = 0.5, train_size=0.5)

K = [1,3,5,7,9,10,25] 
clf = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')

res=[]
# fill in the training and testing data and save as separate variables
for k in K:
    for trainidx, testidx in cv.split(ds.data,ds.target):
        # note that these are sparse matrices
        X_train = ds.data[trainidx] 
        X_test = ds.data[testidx] 
        y_train = ds.target[trainidx]
        y_test = ds.target[testidx]
        
        clf = KNeighborsClassifier(n_neighbors=k, weights='uniform', metric='euclidean')

        clf.fit(X_train,y_train)  # train object
        y_hat = clf.predict(X_test) # get test set precitions

        conf = mt.confusion_matrix(y_test,y_hat)

        row={}
        row["K"]=k
        row["accuracy"]=accuracy_score(y_test, y_hat)
        row["precision"]=precision_score(y_test, y_hat, average="macro")
        row["recall"]=recall_score(y_test, y_hat, average="macro")
        row["f1"]=f1_score(y_test, y_hat, average="macro")
        res.append(row)

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K


acc = 0.2 # overwrite this with the actual accuracy

#=====================================

# print('Accuracy of classifier with %d neighbors is: %.2f'%(K,acc))


<function print>

In [28]:
res

[{'K': 1,
  'accuracy': 0.5981969241647517,
  'precision': 0.6330893808501913,
  'recall': 0.5983987427071629,
  'f1': 0.5971601825785066},
 {'K': 3,
  'accuracy': 0.541629839137352,
  'precision': 0.5875002603514229,
  'recall': 0.5407086363717296,
  'f1': 0.5402138967349099},
 {'K': 5,
  'accuracy': 0.4907194626126922,
  'precision': 0.5868691483059326,
  'recall': 0.491017707097131,
  'f1': 0.5006925635104055},
 {'K': 7,
  'accuracy': 0.4712745271345236,
  'precision': 0.5581614937845362,
  'recall': 0.46872842504532,
  'f1': 0.47192530730039417},
 {'K': 9,
  'accuracy': 0.4765776913558423,
  'precision': 0.552788647073058,
  'recall': 0.47667384453722816,
  'f1': 0.4744401620660593},
 {'K': 10,
  'accuracy': 0.46049142655117553,
  'precision': 0.5513852550902149,
  'recall': 0.4576560129727131,
  'f1': 0.456259871211662},
 {'K': 25,
  'accuracy': 0.3903128866890578,
  'precision': 0.5048737343050946,
  'recall': 0.38929324941454907,
  'f1': 0.38521304824589275}]

The best accuracy is 0.5981969241647517 with 1 neighbors.

**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Enter your answer below:
KD tree does make makes sense to use on sparse data because when data has been partitioned, it's easier to calculate the distance to the closest neighbor and identify the partitions to avoid calculating the distance in because if the distance to the partition is larger than the distance to our closest neighbor, we know that none of the data points inside that partition can be closer. This concept can be used in both dense and sparse data to find the closest neighbor
___

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [34]:
from sklearn.neighbors import NearestCentroid

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)

dis = ['l1', 'l2', 'cosine','manhattan','euclidean' ]
clf = NearestCentroid(metric='euclidean')

res2 = []
# fill in your code here
for d in dis:
    for trainidx, testidx in cv.split(ds.data,ds.target):
        # note that these are sparse matrices
        X_train = ds.data[trainidx] 
        X_test = ds.data[testidx] 
        y_train = ds.target[trainidx]
        y_test = ds.target[testidx]
        
        clf = NearestCentroid(metric=d)

        clf.fit(X_train,y_train)  # train object
        y_hat = clf.predict(X_test) # get test set precitions

        conf = mt.confusion_matrix(y_test,y_hat)

        row={}
        row["Distance"]=d
        row["accuracy"]=accuracy_score(y_test, y_hat)
        row["precision"]=precision_score(y_test, y_hat, average="macro")
        row["recall"]=recall_score(y_test, y_hat, average="macro")
        row["f1"]=f1_score(y_test, y_hat, average="macro")
        res2.append(row)


# print('The best distance metric is: ', p)

/usr/local/lib/python3.7/site-packages/sklearn/neighbors/_nearest_centroid.py:150: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "
/usr/local/lib/python3.7/site-packages/sklearn/neighbors/_nearest_centroid.py:150: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "
/usr/local/lib/python3.7/site-packages/sklearn/neighbors/_nearest_centroid.py:150: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


In [37]:
res2


[{'Distance': 'l1',
  'accuracy': 0.3242001060632844,
  'precision': 0.7321792346143111,
  'recall': 0.32030431824513117,
  'f1': 0.3130919978063357},
 {'Distance': 'l2',
  'accuracy': 0.4196570620470214,
  'precision': 0.46472324254076386,
  'recall': 0.415211234445198,
  'f1': 0.4156310267434942},
 {'Distance': 'cosine',
  'accuracy': 0.47109775499381296,
  'precision': 0.49291313679904947,
  'recall': 0.46535816877426683,
  'f1': 0.4646398784927098},
 {'Distance': 'manhattan',
  'accuracy': 0.1773024571327559,
  'precision': 0.18619003478851054,
  'recall': 0.17496368072860147,
  'f1': 0.15950883414208109},
 {'Distance': 'euclidean',
  'accuracy': 0.40834364504154147,
  'precision': 0.4514386852288026,
  'recall': 0.40380672087083164,
  'f1': 0.40255049797053016}]

The best distance metric is: cosine

___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

In [38]:
print(f'total prob = {20*130000}, prior = 1')

total prob = 2600000, prior = 1


Enter you answer here:

*Double Click to edit*


In [15]:
# Use this space for any calculations you might want to do


___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

Enter you answer here:

Gaussian Naive Bayes is the most useful when doing an analysis involving continuous data. MultinomialNB is often used in text classification where data is represented as word vector counts. While Bernoulli is used for binary or discrete data. If our instances included mostly continuous attributes it would be best to use Gaussian Naive Bayes as a classifier. 
If however, the data was sparse, we might be reluctant to use a Gaussian Naive Bayes model as it will have trouble with all the zero values forcing means close to zero. Given this scenario and the three available models it might be prudent to try the Multinomial Baive Bayes model.
___

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 


A: NB variants are fast because it is a linear classifier. Alpha of zero gives better results for sparce data sets becuase there is no smoothing of the curve.

In [48]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB


clf_mnb = MultinomialNB(alpha=0.0)
clf_bnb = BernoulliNB(alpha=0.0, binarize=0.0)
clf_mnb1 = MultinomialNB(alpha=1.0)
clf_bnb1 = BernoulliNB(alpha=1.0, binarize=0.0)
clf_bnbb1 = BernoulliNB(alpha=0.0, binarize=1.0)

clf_nb = {'mnb0':clf_mnb, 'bnb0':clf_bnb, 'mnb1':clf_mnb1, 'bnb1':clf_bnb1, 'bnbb1':clf_bnbb1}

res3 = []
for name, clf in clf_nb.items():
    for trainidx, testidx in cv.split(ds.data,ds.target):
        # note that these are sparse matrices
        X_train = ds.data[trainidx] 
        X_test = ds.data[testidx] 
        y_train = ds.target[trainidx]
        y_test = ds.target[testidx]
        

        clf.fit(X_train,y_train)  # train object
        y_hat = clf.predict(X_test) # get test set precitions

        conf = mt.confusion_matrix(y_test,y_hat)

        row={}
        row["Name"]=name
        row["accuracy"]=accuracy_score(y_test, y_hat)
        row["precision"]=precision_score(y_test, y_hat, average="macro")
        row["recall"]=recall_score(y_test, y_hat, average="macro")
        row["f1"]=f1_score(y_test, y_hat, average="macro")
        res3.append(row)

            
# print('These classifiers are so fast because...', p)
# print('The alpha values control...', p)


/usr/local/lib/python3.7/site-packages/sklearn/naive_bayes.py:509: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.7/site-packages/sklearn/naive_bayes.py:509: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.7/site-packages/sklearn/naive_bayes.py:509: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
res3

[{'Name': 'mnb0',
  'accuracy': 0.8628248188085558,
  'precision': 0.8645685003703341,
  'recall': 0.8617354833932108,
  'f1': 0.8612738581609956},
 {'Name': 'bnb0',
  'accuracy': 0.8541629839137352,
  'precision': 0.8586943279049004,
  'recall': 0.8518120048336713,
  'f1': 0.8520164659577519},
 {'Name': 'mnb1',
  'accuracy': 0.7021389429025986,
  'precision': 0.8135438385608513,
  'recall': 0.6790569682324433,
  'f1': 0.6760989706851678},
 {'Name': 'bnb1',
  'accuracy': 0.6061516704967297,
  'precision': 0.7242223657815596,
  'recall': 0.5830115227306705,
  'f1': 0.5611009234189634},
 {'Name': 'bnbb1',
  'accuracy': 0.0530316422131872,
  'precision': 0.00265158211065936,
  'recall': 0.05,
  'f1': 0.005036091992613732}]

________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.